# Exemple de classement personnalisé <a class="anchor" id="top"></a>

Dans ce bloc-notes, vous allez choisir un jeu de données et le préparer à être utilisé avec les recommandations par lots d'Amazon Personalize.

1. [Choisir un jeu de données ou une source de données](#source)
1. [Préparer vos données](#prepare)
1. [Créer des groupes de jeux de données et le jeu de données d'interactions](#group_dataset)
1. [Configurer un compartiment S3 et un rôle IAM](#bucket_role)
1. [Importer les données d'interactions](#import)
1. [Créer des solutions](#solutions)
1. [Créer des campagnes](#create)
1. [Interagir avec les campagnes](#interact)
1. [Nettoyer](#cleanup)

## Introduction <a class="anchor" id="intro"></a>

Pour la plupart, les algorithmes d'Amazon Personalize (appelés recettes) cherchent à résoudre différentes tâches, expliquées ici :

1. **HRNN et métadonnées HRNN** – Recommande des articles en fonction des interactions précédentes de l'utilisateur avec les articles.
1. **HRNN-Coldstart** – Recommande de nouveaux articles pour lesquels les données d'interaction ne sont pas encore disponibles.
1. **Classement personnalisé** – Prend une collection d'articles et les ordonne dans l'ordre probable d'intérêt en utilisant une approche de type HRNN.
1. **SIMS (Éléments similaires)** – À partir d'un article donné, recommande d'autres articles avec lesquels les utilisateurs ont également interagi.
1. **Compte de popularité** – Recommande les articles les plus populaires, si HRNN ou métadonnées HRNN n'ont pas de réponse – cela est renvoyé par défaut.

Quel que soit le cas d'utilisation, les algorithmes partagent tous une base d'apprentissage sur les données d'interaction entre l'utilisateur et l'article, qui sont définies par trois attributs fondamentaux :

1. **UserID** – L'utilisateur qui a interagi
1. **ItemID** – Article avec lequel l'utilisateur a interagi
1. **Horodatage** – L'heure à laquelle l'interaction s'est produite

Nous prenons également en charge les types d'événements et les valeurs d'événements définis par :

1. **Type d'événement​​** – Étiquette catégorique d'un événement (parcourir, acheté, évalué, etc.).
1. **Valeur de l'événement** – Une valeur correspondant au type d'événement qui s'est produit. En règle générale, nous recherchons des valeurs normalisées entre 0 et 1 relatives aux types d'événements. Par exemple, s'il y a trois phases pour terminer une transaction (cliqué, ajouté au panier et acheté), alors il y aura une valeur event_value pour chaque phase comme 0,33, 0,66 et 1,0 respectivement.

Les champs type d'événement et valeur d'événement sont des données supplémentaires qui peuvent être utilisées pour filtrer les données envoyées en vue de l'entraînement du modèle de personnalisation. Dans cet exercice particulier, nous n'aurons pas de type d'événement ou de valeur d'événement. 

## Choisir un jeu de données ou une source de données <a class="anchor" id="source"></a>
[Retour au début](#top)

Comme nous l'avons mentionné, les données d'interaction utilisateur-article sont essentielles pour commencer à utiliser le service. Cela signifie que nous devons rechercher les cas d'utilisation qui génèrent ce type de données, dont voici quelques exemples courants :

1. Applications de vidéo à la demande
1. Plateformes d'e-commerce
1. Agrégateurs/plateformes de médias sociaux

Il existe quelques lignes directrices pour définir la portée d'un problème adapté à Personalize. Nous recommandons les valeurs ci-dessous comme point de départ, bien que les [limites officielles](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) soient un peu plus basses.

* Utilisateurs authentifiés
* Au moins 50 utilisateurs uniques
* Au moins 100 articles uniques
* Au moins deux douzaines d'interactions pour chaque utilisateur 

La plupart du temps, il est facile d'y parvenir, et si vous êtes dans une catégorie faible, vous pouvez souvent compenser en ayant un chiffre plus élevé dans une autre catégorie.

En règle générale, vos données n'arriveront pas sous une forme parfaite pour Personalize, et il faudra les modifier afin qu'elles soient structurées correctement. Ce bloc-notes a pour objectif de vous guider dans tout ceci. 

Pour commencer, nous allons utiliser le jeu de données [Last.FM](https://grouplens.org/datasets/hetrec-2011/). Il s'agit d'enregistrements du comportement d'écoute musicale de ses utilisateurs. Les données correspondent à nos directives avec un grand nombre d'utilisateurs, d'articles et d'interactions.

Tout d'abord, vous allez télécharger le jeu de données et le décompresser dans un nouveau dossier en utilisant le code ci-dessous.

In [4]:
data_dir = "data"
!mkdir $data_dir
!cd $data_dir && wget http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!cd $data_dir && unzip hetrec2011-lastfm-2k.zip

--2020-05-26 20:45:51--  http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2589075 (2.5M) [application/zip]
Saving to: ‘hetrec2011-lastfm-2k.zip’

hetrec2011-lastfm-2 100%[===================>]   2.47M  10.2MB/s    in 0.2s    

2020-05-26 20:45:51 (10.2 MB/s) - ‘hetrec2011-lastfm-2k.zip’ saved [2589075/2589075]

Archive:  hetrec2011-lastfm-2k.zip
  inflating: user_friends.dat        
  inflating: user_taggedartists.dat  
  inflating: user_taggedartists-timestamps.dat  
  inflating: artists.dat             
  inflating: readme.txt              
  inflating: tags.dat                
  inflating: user_artists.dat        


Examinez les fichiers de données que vous avez téléchargés.

In [5]:
!ls $data_dir

artists.dat		  tags.dat	    user_taggedartists.dat
hetrec2011-lastfm-2k.zip  user_artists.dat  user_taggedartists-timestamps.dat
readme.txt		  user_friends.dat


Pour l'instant, nous avons très peu d'informations sur les données, si ce n'est qu'il semble exister un grand nombre de fichiers .dat et un fichier README. L'ouverture du fichier README nous renseignera sur la structure globale de ces données. C'est une étape que vous pouvez probablement ignorer avec des données personnalisées, sauf si la source de données provient d'une équipe externe

D'après le fichier README, nous constatons qu'il existe plusieurs types d'interactions dans ce jeu de données. Les interactions entre les utilisateurs qui se marquent mutuellement comme amis, les interactions des utilisateurs qui écoutent des artistes et les interactions des identifications attribuées aux utilisateurs et aux artistes.

Dans ce cas, nous nous concentrons sur les utilisateurs, les artistes et les interactions d'écoute. Nous avons 1 892 utilisateurs, 17 632 artistes (nos articles dans ce cas) et 92 834 interactions entre les utilisateurs et les artistes écoutés. Cela suffit amplement pour commencer à utiliser Personalize.

Continuez à lire le fichier README jusqu'à la section `Files`. La plupart des fichiers du jeu de données ne sont pas pertinents pour nous, mais ce fichier `users_artists.dat` semble prometteur. La section `Data format` du fichier README fournit plus de détails sur le contenu du fichier. C'est là que nous rencontrons notre premier problème.

| userID | artistID | poids  |
|--------|----------|---------|
| 2      | 51       | 13883   |

Bien qu'il existe des données d'interaction entre les utilisateurs et les artistes qu'ils écoutent, ces interactions sont stockées sous forme de poids et non d'horodatage. Nous avons besoin des données d'interaction utilisateur-article-horodatage pour Amazon Personalize. 

Si vous examinez à nouveau les fichiers dans le jeu de données, vous devez constater que `users_taggedartists-timestamps.dat` contient des données d'horodatage. Et si nous utilisions le comportement d'identification comme données d'interaction, au lieu du comportement d'écoute ? Pouvons-nous supposer qu'un utilisateur qui identifie un artiste est une indication d'un sentiment positif ? Normalement, vous discuterez avec votre client, ou avec quelqu'un qui a des connaissances dans le domaine, pour comprendre si cette interaction est adaptée au cas d'utilisation que vous voulez résoudre. Pour l'instant, nous supposerons que le comportement d'identification est adapté à nos besoins. 

Le schéma pour `user_taggedartists-timestamps.dat` est :

| userID | artistID | tagID | timestamp     |
|--------|----------|-------|---------------|
| 2      | 52       | 13    | 1238536800000 |

Si nous supprimons l'attribut `tagID`, nous avons exactement le format dont nous avons besoin pour Amazon Personalize.

## Préparer vos données <a class="anchor" id="prepare"></a>
[Retour au début](#top)

Chargez ensuite les données et assurez-vous qu'elles sont en bon état, puis enregistrez-les dans un CSV où elles sont prêtes à être utilisées avec Amazon Personalize.

Pour commencer, importez une collection de bibliothèques Python couramment utilisées en science des données.

In [6]:
import time
from time import sleep
import json
from datetime import datetime
import numpy as np
import boto3
import pandas as pd

Ensuite, ouvrez le fichier de données et examinez les premières lignes.

In [7]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat')
original_data.head(5)

,userID artistID tagID timestamp
0,2\t52\t13\t1238536800000
1,2\t52\t15\t1238536800000
2,2\t52\t18\t1238536800000
3,2\t52\t21\t1238536800000
4,2\t52\t41\t1238536800000


À l'évidence, les données ne se sont pas chargées correctement. Le délimiteur par défaut des fichiers CSV (comma-separated value) est la virgule (`,`), mais dans ce cas, le fichier a été enregistré avec des caractères de tabulation (`\t`). Spécifions donc le délimiteur correct et essayons à nouveau de charger les données.

In [8]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat', delimiter='\t')
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1238536800000
1,2,52,15,1238536800000
2,2,52,18,1238536800000
3,2,52,21,1238536800000
4,2,52,41,1238536800000


C'est mieux. Maintenant que les données ont été chargées en mémoire, extrayons quelques informations supplémentaires. Tout d'abord, calculez quelques statistiques de base à partir des données.

In [9]:
original_data.describe()

,userID,artistID,tagID,timestamp
count,186479.000000,186479.000000,186479.000000,1.864790e+05
mean,1035.600137,4375.845328,1439.582913,1.239204e+12
std,622.461272,4897.789595,2775.340279,4.299091e+10
min,2.000000,1.000000,1.000000,-4.287204e+11
25%,488.000000,686.000000,79.000000,1.209593e+12
50%,1021.000000,2203.000000,195.000000,1.243807e+12
75%,1624.000000,6714.000000,887.000000,1.275343e+12
max,2100.000000,18744.000000,12647.000000,1.304941e+12


Cela montre que nous avons une bonne gamme de valeurs pour `userID` et `artistID`. Ensuite, il est toujours bon de vérifier le format des données.

In [10]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186479 entries, 0 to 186478
Data columns (total 4 columns):
userID       186479 non-null int64
artistID     186479 non-null int64
tagID        186479 non-null int64
timestamp    186479 non-null int64
dtypes: int64(4)
memory usage: 5.7 MB


À partir de là, vous pouvez voir qu'il y a un total de 186 479 entrées dans le jeu de données, avec 4 colonnes, et que chaque cellule est stockée au format int64.

Le format int64 est clairement adapté à `userID` et `artistID`. Cependant, nous effectuer une analyse plus approfondie pour comprendre les horodatages dans les données. Pour utiliser Amazon Personalize, vous devez enregistrer les horodatages au format [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time).

Actuellement, les valeurs d'horodatage ne sont pas lisibles par l'homme. Prenons donc une valeur d'horodatage arbitraire et voyons comment l'interpréter.

In [11]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000000


ValueError: year 41132 is out of range

Oups ! Pour cette valeur d'horodatage particulière, le code a rendu l'année 41 132. C'est un peu loin dans le futur pour nous, et, clairement, ce n'est la bonne façon d'analyser les données. Nous avons besoin d'une deuxième tentative.

JavaScript enregistre le temps en millisecondes, et il s'agit d'une collection de données provenant d'une application web. Divisons donc la valeur de l'horodatage par 1 000 avant d'appliquer notre code.

In [12]:
arb_time_stamp = arb_time_stamp/1000
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

2009-02-28 23:00:00


Le mois de février 2009 semble beaucoup plus réaliste pour notre jeu de données. Nous n'avons pas besoin d'horodatages lisible par l'homme pour utiliser Amazon Personalize, mais nous tenons à ce que les dates soient réalistes. Nous allons donc transformer chaque horodatage du jeu de données pour l'éloigner du format JavaScript en millisecondes. 

In [13]:
original_data.timestamp = original_data.timestamp / 1000
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1.238537e+09
1,2,52,15,1.238537e+09
2,2,52,18,1.238537e+09
3,2,52,21,1.238537e+09
4,2,52,41,1.238537e+09


Effectuez un test d'intégrité du jeu de données transformé en choisissant un horodatage arbitraire et en le transformant en un format lisible par l'homme.

In [14]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000.0
2009-02-28 23:00:00


Cette date a du sens en tant qu'horodatage. Nous pouvons donc continuer à formater le reste des données. Rappelez-vous que les données dont nous avons besoin sont des données d'interaction utilisateur-article, qui sont `userID`, `artistID` et `timestamp` dans ce cas. Notre jeu de données comporte une colonne supplémentaire, `tagID`, qui peut être supprimée du jeu de données.

In [15]:
interactions_df = original_data.copy()
interactions_df = interactions_df[['userID', 'artistID', 'timestamp']]
interactions_df.head()

,userID,artistID,timestamp
0,2,52,1.238537e+09
1,2,52,1.238537e+09
2,2,52,1.238537e+09
3,2,52,1.238537e+09
4,2,52,1.238537e+09


Après avoir manipulé les données, vérifiez toujours si le format des données a changé.

In [16]:
interactions_df.dtypes

userID         int64
artistID       int64
timestamp    float64
dtype: object

Dans ce cas, le format int64 de la colonne horodatage a été remplacé par float64. Par conséquent, rétablissons le format int64.

In [17]:
interactions_df.astype({'timestamp': 'int64'}).dtypes

userID       int64
artistID     int64
timestamp    int64
dtype: object

 Amazon Personalize utilise des noms de colonne par défaut pour les utilisateurs, les articles et l'horodatage. Ces noms de colonnes par défaut sont `USER_ID`, `ITEM_ID` ET `TIMESTAMP`. La dernière modification apportée au jeu de données consiste donc à remplacer les en-têtes de colonnes existants par les en-têtes par défaut.

In [18]:
interactions_df.rename(columns = {'userID':'USER_ID', 'artistID':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

Voilà ! À ce stade, les données sont prêtes à être utilisées et il ne nous reste plus qu'à les enregistrer dans un fichier CSV.

In [19]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Créer des groupes de jeux de données et le jeu de données d'interactions <a class="anchor" id="group_dataset"></a>
[Retour au début](#top)

Le plus haut niveau d'isolation et d'abstraction avec Amazon Personalize est le *groupe de jeux de données*. Toute information stockée dans l'un de ces groupes de jeux de données n'a aucun impact sur un autre groupe de données ou sur les modèles créés à partir de celui-ci – ils sont entièrement indépendants. Vous pouvez ainsi réaliser de nombreuses expériences, et c'est en partie grâce à cela que vos modèles restent privés et ne sont entraînés que sur vos données. 

Avant d'importer les données préparées antérieurement, il faut créer un groupe de jeux de données et y ajouter un jeu de données qui gère les interactions.

Les groupes de jeux de données peuvent comporter les types d'informations suivants :

* Interactions utilisateur-article
* Flux d'événements (interactions en temps réel)
* Métadonnées de l'utilisateur
* Métadonnées de l'article

Avant de créer le groupe de jeux de données et le jeu de données pour nos données d'interaction, vérifions que votre environnement peut communiquer avec Amazon Personalize.

In [20]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Créer le groupe de jeux de données

La cellule suivante va créer un nouveau groupe de jeux de données avec le nom `personalize-poc-lastfm`.

In [21]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-ranking-dsg"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/personalize-ranking-dsg",
  "ResponseMetadata": {
    "RequestId": "386224ac-8410-4f64-ba4a-460ba12607c7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:46:27 GMT",
      "x-amzn-requestid": "386224ac-8410-4f64-ba4a-460ba12607c7",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Pour pouvoir utiliser le groupe de jeux de données, il doit être actif. Cela peut prendre une ou deux minutes. Exécutez la cellule ci-dessous et attendez qu'elle affiche le statut ACTIF. Il vérifie l'état du groupe de jeux de données toutes les secondes, jusqu'à un maximum de 3 heures.

In [22]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


Maintenant que vous avez un groupe de jeux de données, vous pouvez créer un jeu de données pour les données d'interaction.

### Créer le jeu de données

Tout d'abord, définissez un schéma afin d'indiquer à Amazon Personalize le type de jeu de données que vous chargez. Plusieurs mots-clés réservés et obligatoires sont requis dans le schéma, en fonction du type de jeu de données. Des informations plus détaillées peuvent être retrouvées dans la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Ici, vous allez créer un schéma pour les données d'interactions, qui nécessite les champs `USER_ID`, `ITEM_ID` et `TIMESTAMP`  Ils doivent être définis dans le schéma en respectant l'ordre dans lequel ils apparaissent dans le jeu de données.

In [23]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ranking-interactions",
    schema = json.dumps(interactions_schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/personalize-ranking-interactions",
  "ResponseMetadata": {
    "RequestId": "d3fb42c4-9a3c-45a4-a842-adf60ef03906",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:47:58 GMT",
      "x-amzn-requestid": "d3fb42c4-9a3c-45a4-a842-adf60ef03906",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Une fois le schéma créé, vous pouvez créer un jeu de données dans le groupe de jeux de données. Il est à noter que cette opération ne charge pas encore les données. Cela arrivera quelques étapes plus tard.

In [24]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ranking-ds",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/personalize-ranking-dsg/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "98a3c225-31aa-49be-936f-f633aa6314d0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:48:02 GMT",
      "x-amzn-requestid": "98a3c225-31aa-49be-936f-f633aa6314d0",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Configurer un compartiment S3 et un rôle IAM <a class="anchor" id="bucket_role"></a>
[Retour au début](#top)

Pour l'instant, nous avons téléchargé, manipulé et enregistré les données sur l'instance Amazon EBS associée à l'instance qui exécute ce bloc-notes Jupyter. Toutefois, Amazon Personalize aura besoin d'un compartiment S3 pour servir de source à vos données, ainsi que de rôles IAM pour accéder à ce compartiment. Mettons tout cela en place.

Utilisez les métadonnées stockées sur l'instance sous-jacente de ce bloc-notes Amazon SageMaker pour déterminer la région dans laquelle il fonctionne. Si vous utilisez un bloc-notes Jupyter en dehors d'Amazon SageMaker, définissez simplement la région sous forme de chaîne comme suit. Le compartiment Amazon S3 doit se trouver dans la même région que les ressources Amazon Personalize que nous avons créées jusqu'à présent.

In [25]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Les noms des compartiments Amazon S3 sont uniques au niveau mondial. Pour créer un nom de compartiment unique, le code ci-dessous ajoutera la chaîne de caractères `personalizepoc` à votre numéro de compte AWS. Puis il crée un compartiment avec ce nom dans la région découverte dans la cellule précédente.

In [26]:
s3 = boto3.client('s3')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-ranking-demo-"+   suffix        # replace with the name of your S3 bucket
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

personalize-ranking-demo-41600


### Charger les données vers S3

Maintenant que votre compartiment Amazon S3 a été créé, chargez le fichier CSV de nos données d'interaction utilisateur-article. 

In [27]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### Définir la politique du compartiment S3
Amazon Personalize doit pouvoir lire le contenu de votre compartiment S3. Il faut donc ajouter une politique de compartiment qui l'autorise.

In [28]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '61279AD0FB422CD2',
  'HostId': 'pvAc3tsFJjLUU5BTudF9gokRma/yRh+uH3TdrVXk3/xoLlHsOev2XPv8KDc+TusZphyzL/TDUvE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'pvAc3tsFJjLUU5BTudF9gokRma/yRh+uH3TdrVXk3/xoLlHsOev2XPv8KDc+TusZphyzL/TDUvE=',
   'x-amz-request-id': '61279AD0FB422CD2',
   'date': 'Tue, 26 May 2020 20:48:18 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Créer un rôle IAM

Amazon Personalize doit pouvoir assumer des rôles dans AWS, afin de disposer des autorisations nécessaires pour exécuter certaines tâches. Créons un rôle IAM et ajoutons-lui les politiques requises. Le code ci-dessous associe des politiques très permissives ; veuillez utiliser des politiques plus restrictives pour toute application de production.

In [29]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRanking"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeRoleRanking


## Importer les données d'interactions <a class="anchor" id="import"></a>
[Retour au début](#top)

Précédemment, vous avez créé le groupe de jeux de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données du compartiment S3 dans le jeu de données Amazon Personalize. 

In [30]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ranking-example",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/personalize-ranking-example",
  "ResponseMetadata": {
    "RequestId": "7eab862d-9724-41dc-87d3-7455b4f16ead",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:51:57 GMT",
      "x-amzn-requestid": "7eab862d-9724-41dc-87d3-7455b4f16ead",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Pour pouvoir utiliser le jeu de données, la tâche d'importation doit être active. Exécutez la cellule ci-dessous et attendez qu'elle affiche le statut ACTIF. Il vérifie l'état de la tâche d'importation toutes les secondes, jusqu'à un maximum de 3 heures.

L'importation des données peut prendre un certain temps, en fonction de la taille du jeu de données. Dans cet atelier, l'importation des données devrait durer environ 15 minutes.

In [31]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 50 ms, sys: 15.1 ms, total: 65 ms
Wall time: 13min


Lorsque l'importation de jeux de données est active, vous êtes prêt à commencer à construire des modèles avec SIMS, Classement Personnalisé, Compteur de Popularité et HRNN. Ce processus se poursuivra dans d'autres blocs-notes. Exécutez la cellule ci-dessous avant de passer au stockage de quelques valeurs à utiliser dans les prochains blocs-notes.

## Créer des solutions <a class="anchor" id="solutions"></a>
[Retour au début](#top)

Dans ce bloc-notes, vous allez créer des solutions avec la recette suivante :

1. HRNN


Dans Amazon Personalize, une variation spécifique d'un algorithme est appelée une recette. Des recettes différentes conviennent à des situations différentes. Un modèle entraîné s'appelle une solution, et chaque solution peut avoir de nombreuses versions qui se rapportent à un volume de données spécifique au moment où le modèle a été entraîné.

Tout d'abord, nous allons énumérer toutes les recettes qui sont prises en charge. Cela vous permettra d'en sélectionner une et de l'utiliser pour créer votre modèle.

In [32]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   's

### Classement personnalisé

Le classement personnalisé est une application intéressante de HRNN. Au lieu de recommander uniquement ce qui est le plus probable pour l'utilisateur concerné, cet algorithme prend en compte un utilisateur ainsi qu'une liste d'articles. Les articles sont ensuite restitués par ordre de pertinence la plus probable pour l'utilisateur. Le cas d'utilisation ici vise le filtrage par genre en particulier, ou lorsque vous avez une vaste collection que vous souhaitez mieux ordonner pour un utilisateur particulier.

Pour notre cas d'utilisation, en utilisant les données de LastFM, nous pourrions supposer qu'une maison de disques en particulier nous paie pour recommander ses artistes à nos utilisateurs dans le cadre d'une promotion spéciale. Nous savons donc quelle liste d'artistes nous voulons recommander, mais nous cherchons à savoir lesquels de ces artistes chaque utilisateur appréciera le plus. Nous utiliserions le classement personnalisé pour réorganiser la liste des artistes pour chaque utilisateur, en fonction de son historique d'étiquetage. 

Comme la dernière fois, nous commençons par sélectionner la recette.

In [33]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Créer la solution

Comme pour la solution précédente, commencez par créer la solution. Même si vous fournissez le jeu de données ARN dans cette étape, le modèle n'est pas encore entraîné. Considérez cela comme un identifiant plutôt qu'un modèle entraîné.

In [34]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-ranking",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking",
  "ResponseMetadata": {
    "RequestId": "ba5cdcc0-de91-4bf0-a06d-dde91ff1b6ac",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:04:58 GMT",
      "x-amzn-requestid": "ba5cdcc0-de91-4bf0-a06d-dde91ff1b6ac",
      "content-length": "89",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Créer la version de solution

Une fois que vous avez une solution, vous devez créer une version afin d'achever l'entraînement du modèle. L'entraînement peut prendre un certain temps, plus de 25 minutes, et une moyenne de 35 minutes pour cette recette avec notre jeu de données. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Toutefois, cette tâche bloquerait l'exécution d'autres cellules, et l'objectif ici est de créer de nombreux modèles et de les déployer rapidement. Nous allons donc mettre en place la boucle while pour toutes les solutions plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [35]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [36]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking/0f00a819",
  "ResponseMetadata": {
    "RequestId": "b07c347c-48aa-4650-86f6-53197351a62f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:04:58 GMT",
      "x-amzn-requestid": "b07c347c-48aa-4650-86f6-53197351a62f",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Affichez le statut de création de la solution

Voici, comme promis, comment afficher les mises à jour du statut dans la console :

* La console AWS devrait déjà être ouverte dans un autre onglet du navigateur, après l'ouverture de cette instance du bloc-notes. 
* Passez à cet onglet et recherchez en haut le service `Personalize`, puis référez-vous à cette page de service. 
* Cliquez sur `View dataset groups`.
* Cliquez sur le nom de votre groupe de jeux de données, très probablement un nom contenant POC.
* Cliquez sur `Solutions and recipes`.
* Vous voyez maintenant la liste de toutes les solutions que vous avez créées ci-dessus, y compris une colonne avec le statut des versions de la solution. Une fois qu'elle est `Active`, votre solution est prête à être examinée. Elle est également en mesure d'être déployée.

Ou exécutez simplement la cellule ci-dessous pour surveiller le statut de création des versions de la solution.

In [38]:
in_progress_solution_versions = [
    rerank_solution_version_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

Build succeeded for arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking/0f00a819


## Créer des campagnes <a class="anchor" id="create"></a>
[Retour au début](#top)

Une campagne est une version de la solution hébergée, un point de terminaison que vous pouvez interroger pour obtenir des recommandations. La tarification est fixée en estimant la capacité de débit (demandes de personnalisation de l'utilisateur par seconde). Lors du déploiement d'une campagne, vous définissez une valeur minimale de débit par seconde (TPS). Ce service, comme beaucoup d'autres au sein d'AWS, évoluera automatiquement en fonction de la demande. Néanmoins, si la latence est stratégique, vous voudrez peut-être prendre des dispositions pour une demande plus importante. Pour cette POC et cette démo, tous les seuils de débit minimum sont définis à 1. Pour plus d'informations, reportez-vous à la [page de tarification](https://aws.amazon.com/personalize/pricing/).

Commençons à déployer les campagnes.

### Classement personnalisé

Déployez une campagne pour la version de votre solution de classement personnalisée. Le déploiement d'une campagne peut prendre environ 10 minutes. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Cependant, la tâche bloquerait l'exécution d'autres cellules, et le but ici est de créer de multiples campagnes. Nous allons donc mettre en place la boucle while pour toutes les campagnes plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [39]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:144386903708:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "61fd83dd-e409-4433-a2b9-667aecdecb0a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:48:55 GMT",
      "x-amzn-requestid": "61fd83dd-e409-4433-a2b9-667aecdecb0a",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Afficher le statut de création de la campagne

Voici, comme promis, comment afficher les mises à jour du statut dans la console :

* La console AWS devrait déjà être ouverte dans un autre onglet du navigateur, après l'ouverture de cette instance du bloc-notes. 
* Passez à cet onglet et recherchez en haut le service `Personalize`, puis référez-vous à cette page de service. 
* Cliquez sur `View dataset groups`.
* Cliquez sur le nom de votre groupe de jeux de données, très probablement un nom contenant POC.
* Cliquez sur `Campaigns`.
* Vous verrez maintenant une liste de toutes les campagnes que vous avez créées ci-dessus, y compris une colonne avec le statut de la campagne. Une fois qu'elle est `Active`, votre campagne est prête à être interrogée.

Ou exécutez simplement la cellule ci-dessous pour surveiller le statut de création de la campagne.

In [41]:
in_progress_campaigns = [
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:144386903708:campaign/personalize-poc-rerank


## Interagir avec les campagnes <a class="anchor" id="interact"></a>
[Retour au début](#top)

Maintenant que toutes les campagnes sont déployées et actives, nous pouvons obtenir des recommandations via un appel API. Chacune des campagnes est basée sur une recette différente, qui se comporte de manière légèrement différente, car elle répond à des cas d'utilisation différents. Nous aborderons chaque campagne dans un ordre différent de celui utilisé dans les bloc-notes précédents, afin de traiter les éventuelles complexités par ordre croissant (c'est-à-dire le plus simple en premier).

Tout d'abord, créons une fonction de prise en charge pour faciliter la compréhension des résultats renvoyés par une campagne Personalize. Personalize renvoie uniquement un `item_id`. Cette méthode est très pratique pour conserver des données compactes, mais elle signifie que vous devez interroger une base de données ou une table de consultation pour obtenir un résultat lisible par l'homme pour les bloc-notes. Nous allons créer une fonction d'aide pour renvoyer un résultat lisible par l'homme à partir du jeu de données LastFM.

Commencez par charger le jeu de données que nous pouvons utiliser pour notre table de consultation.

In [42]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)

# Render some sample data
items_df.head(5)

,name,url,pictureURL
id,,,
1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


En définissant la colonne ID comme colonne d'index, il est aisé de trouver un artiste en recherchant simplement l'ID.

In [43]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

Earth, Wind & Fire


Cette méthode est acceptable. Cependant, il serait fastidieux de la répéter partout dans notre code. C'est pourquoi la fonction ci-dessous va la supprimer.

In [44]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

Testons à présent quelques valeurs simples pour vérifier notre système de détection d'erreurs.

In [45]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

Earth, Wind & Fire
Error obtaining artist
Error obtaining artist


Super ! Nous disposons désormais d'un moyen de présentation des résultats. 

In [47]:
users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
# Render some sample data
users_df.head(5)

,artistID,weight
userID,,
2,51,13883
2,52,11690
2,53,11351
2,54,10300
2,55,8983


### Classement personnalisé

Le cas d'utilisation principal du classement personnalisé consiste à traiter une collection d'articles et à les présenter par ordre de priorité ou d'intérêt probable pour un utilisateur. Pour le démontrer, nous avons besoin d'un utilisateur aléatoire et d'une collection aléatoire de 25 éléments.

In [48]:
rerank_user = users_df.sample(1).index.tolist()[0]
rerank_items = items_df.sample(25).index.tolist()

Créez maintenant un tableau de données qui montre les données d'entrée.

In [49]:
rerank_list = []
for item in rerank_items:
    artist = get_artist_by_id(item)
    rerank_list.append(artist)
rerank_df = pd.DataFrame(rerank_list, columns = [rerank_user])
rerank_df

,1303
0,Priestess
1,Пелагея
2,Cosmic Gate
3,Mujuice
4,Prism
5,WIZO
6,Love of Lesbian
7,Mayhem
8,Taking Dawn
9,Universum


Lancez ensuite l'appel d'API de classement personnalisé.

In [50]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': 'fd2ea40f-11f5-461c-b678-6a6a08bdb416',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Tue, 26 May 2020 22:02:55 GMT',
   'x-amzn-requestid': 'fd2ea40f-11f5-461c-b678-6a6a08bdb416',
   'content-length': '1442',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '1047', 'score': 0.1730128},
  {'itemId': '1046', 'score': 0.1639514},
  {'itemId': '1254', 'score': 0.1101557},
  {'itemId': '18692', 'score': 0.0762716},
  {'itemId': '5783', 'score': 0.0678702},
  {'itemId': '13434', 'score': 0.067667},
  {'itemId': '8915', 'score': 0.0616803},
  {'itemId': '1305', 'score': 0.0322442},
  {'itemId': '8543', 'score': 0.0297265},
  {'itemId': '11979', 'score': 0.0286533},
  {'itemId': '6820', 'score': 0.025196},
  {'itemId': '4154', 'score': 0.0246835},
  {'itemId': '2312', 'score': 0.0228279},
  {'itemId': '2158', 'score': 0.0179717},
  {'itemId': '2036', 'score': 0.0176177},
 

Ajoutez à présent les articles reclassés dans une deuxième colonne du tableau de données original, pour une comparaison parallèle.

In [51]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    artist = get_artist_by_id(item['itemId'])
    ranked_list.append(artist)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,1303,Re-Ranked
0,Priestess,McFly
1,Пелагея,New Found Glory
2,Cosmic Gate,Mayhem
3,Mujuice,Farid Farjad
4,Prism,Priestess
5,WIZO,Пелагея
6,Love of Lesbian,How to Destroy Angels
7,Mayhem,Cosmic Gate
8,Taking Dawn,Prism
9,Universum,Murder by Death


Vous pouvez constater ci-dessus comment chaque entrée a été réorganisée en fonction de la compréhension du modèle par l'utilisateur. C'est une tâche courante lorsque vous avez une collection d'éléments à montrer à un utilisateur, une liste de promotions par exemple, ou si vous filtrez sur une catégorie et que vous voulez montrer les éléments les plus susceptibles d'être bons.

## Nettoyage des campagnes

Commencez par nettoyer les campagnes. Vous devez supprimer les campagnes avant de pouvoir supprimer les versions de la solution sur lesquelles elles sont basées. 

Le code ci-dessous énumérera toutes les campagnes de votre compte.

In [ ]:
paginator = personalize.get_paginator('list_campaigns')
for paginate_result in paginator.paginate():
    for campaign in paginate_result["campaigns"]:
        print(campaign["campaignArn"])

Examinez la liste des ARN pour déterminer les campagnes que vous voulez supprimer. Utilisez ensuite le code ci-dessous pour supprimer la campagne en insérant l'ARN.

In [ ]:
personalize.delete_campaign(
    campaignArn = "INSERT ARN HERE"
)

## Nettoyer les solutions

Nettoyez ensuite les solutions. Le code ci-dessous énumérera toutes les solutions dans votre compte.

In [ ]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

Consultez la liste des ARN pour déterminer la solution à supprimer. Utilisez ensuite le code ci-dessous pour le supprimer en insérant l'ARN.

In [ ]:
personalize.delete_solution(
    solutionArn = "INSERT ARN HERE"
)

## Nettoyer les jeux de données

Nettoyez ensuite les jeux de données. Le code ci-dessous énumérera tous les jeux de données de votre compte.

In [ ]:
paginator = personalize.get_paginator('list_datasets')
for paginate_result in paginator.paginate():
    for datasets in paginate_result["datasets"]:
        print(datasets["datasetArn"])

Consultez la liste des ARN pour déterminer le jeu de données à supprimer. Utilisez ensuite le code ci-dessous pour le supprimer en insérant l'ARN.

In [ ]:
personalize.delete_dataset(
    datasetArn = "INSERT ARN HERE"
)

## Nettoyer les schémas

Nettoyez ensuite les schémas. Le code ci-dessous énumérera tous les schémas de votre compte.

In [ ]:
paginator = personalize.get_paginator('list_schemas')
for paginate_result in paginator.paginate():
    for schema in paginate_result["schemas"]:
        print(schema["schemaArn"])

Consultez la liste des ARN pour déterminer le schéma à supprimer. Utilisez ensuite le code ci-dessous pour le supprimer en insérant l'ARN.

In [ ]:
personalize.delete_schema(
    schemaArn = "INSERT ARN HERE"
)

## Nettoyer le groupe de jeux de données

Enfin, nettoyez le groupe de jeux de données. Le code ci-dessous énumérera tous les groupes de jeux de données de votre compte.

In [ ]:
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        print(dataset_group["datasetGroupArn"])

Consultez la liste des ARN pour déterminer le groupe de jeux de données à supprimer. Utilisez ensuite le code ci-dessous pour le supprimer en insérant l'ARN.

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = "INSERT ARN HERE"
)

## Nettoyer le compartiment S3 et le rôle IAM

Vous pouvez éventuellement supprimer le rôle IAM et le compartiment S3 que nous avons créés pendant l'atelier.

Commencez par répertorier tous les rôles IAM de votre compte à l'aide du code ci-dessous.

In [ ]:
iam = boto3.client('iam')

paginator = iam.get_paginator('list_roles')
for paginate_result in paginator.paginate():
    for roles in paginate_result["Roles"]:
        print(roles["RoleName"])

Identifiez le nom du rôle que vous souhaitez supprimer.

Vous ne pouvez pas supprimer un rôle IAM auquel des politiques sont encore attachées. Donc, après avoir identifié le rôle pertinent, répertorions la liste des politiques attachées au rôle.

In [ ]:
iam.list_attached_role_policies(
    RoleName = "INSERT ROLE NAME HERE"
)

Vous devez détacher les politiques dans le résultat ci-dessus en utilisant le code ci-dessous. Répétez l'opération pour chaque politique attachée.

In [ ]:
iam.detach_role_policy(
    RoleName = "INSERT ROLE NAME HERE",
    PolicyArn = "INSERT ARN HERE"
)

Enfin, vous devriez être en mesure de supprimer le rôle IAM.

In [ ]:
iam.delete_role(
    RoleName = "INSERT ROLE NAME HERE"
)

Pour supprimer un compartiment S3, il doit d'abord être vide. La façon la plus facile de supprimer un compartiment S3 est simplement de naviguer vers S3 dans la console AWS, de supprimer les objets du compartiment, puis de supprimer le compartiment S3 lui-même.